In [0]:
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
from torch import optim
from torch import nn
import copy

In [2]:
device = torch.device('cuda')
print(device.type)

cuda


In [0]:
training_transforms = transforms.Compose([
                             transforms.RandomResizedCrop(256),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

testing_transforms = transforms.Compose([
                            transforms.RandomResizedCrop(256),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])


In [4]:
train_data = torchvision.datasets.CIFAR10(root = './data',train= True,download = True,transform= training_transforms)
test_data = torchvision.datasets.CIFAR10(root='./data',train = True,download = True,transform = testing_transforms)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
train_dataloader = torch.utils.data.DataLoader(dataset = train_data,batch_size = 50,shuffle = True)
test_dataloader = torch.utils.data.DataLoader(dataset=test_data,batch_size = 50)

In [6]:
print(len(train_dataloader))

1000


In [0]:
#Building ALEXNET from scratch
class Alex(nn.Module):
  def __init__(self):
    super(Alex,self).__init__()
    self.features = nn.Sequential(
                    nn.Conv2d(3,64,kernel_size = 11,stride =4,padding =2),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size = 3,stride = 2),
                    nn.Conv2d(64,192,kernel_size = 5,padding = 2),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3,stride=2),
                    nn.Conv2d(192,384,kernel_size=3,padding=1),
                    nn.ReLU(),
                    nn.Conv2d(384,256,kernel_size = 3,padding = 1),
                    nn.ReLU(),
                    nn.Conv2d(256,256,kernel_size = 3,padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size = 3,stride = 2))
    self.avgpool = nn.AdaptiveAvgPool2d((6,6))
    self.classifier = nn.Sequential(
                    nn.Dropout(),
                    nn.Linear(256*6*6,4096),
                    nn.ReLU(),
                    nn.Dropout(),
                    nn.Linear(4096,4096),
                    nn.ReLU(),
                    nn.Linear(4096,10))
  def forward(self,x):
      x = self.features(x)
      x = self.avgpool(x)
      x = torch.flatten(x,1)
      x = self.classifier(x)
      return x

    

In [0]:
train_data_iter = iter(train_dataloader)
test_data_iter = iter(test_dataloader)

In [0]:
model = Alex()
model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
opti = optim.SGD(model.parameters(),lr = 0.15)
#print(model)

In [0]:
epochs = 100
epoch_loss = []
min_loss = 10000
for i in range(epochs):
  for images,labels in train_dataloader:
    opti.zero_grad()
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    loss = loss_fn(outputs,labels)
    loss.backward()
    opti.step()
    
    del images,labels,outputs    #deleting the images, outputs and labels to free the cache memory on GPU
    torch.cuda.empty_cache()
    
    if loss.item() < abs(min_loss):
      min_loss = loss.item()
      best_model = copy.deepcopy(model.state_dict)
      print(f'Min loss is {min_loss}')
  epoch_loss.append(loss.item())

plt.plot(epoch_loss)
plt.show()



In [0]:
def accuracy(model,data):
  total = 0,correct = 0
  for images,labels in data:
    outputs = model(images)
    output = torch.argmax(outputs,1)
    for i in range(len(labels)):
      if labels[i] == output[i]:
        correct +=1
      total +=1
  return 100*(correct/total)
print(accuracy(best_model,train_data_iter))
print(accuracy(best_model,test_data_iter))